In [2]:
import numpy as np
import pandas as pd
import torch
import string
import re
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from sklearn.preprocessing import LabelEncoder
from torchtext.vocab import build_vocab_from_iterator
import torchtext.vocab as vocab

In [3]:
training_df = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv', header=None)
training_df.shape

(74682, 4)

In [ ]:
training_df.sample(5)

In [4]:
print(f'Shape before dropping nulls {training_df.shape}')
training_df = training_df.dropna()
print(f'Shape after dropping nulls {training_df.shape}')

Shape before dropping nulls (74682, 4)
Shape after dropping nulls (73996, 4)


In [5]:
training_df.loc[:, 3] = training_df.loc[:, 3].str.replace(re.escape(string.punctuation), ' ', regex=True)
training_df.loc[:, 3] = training_df.loc[:, 3].str.replace(',', ' ', regex=True)

In [6]:
input_texts = training_df[3].tolist()
print(len(input_texts))

tokenizer = get_tokenizer('basic_english')

tokenized_texts = [tokenizer(text) for text in input_texts]
# tokenized_texts = tokenized_texts



# flattened_list = sum(tokenized_texts, [])
flattened_list = [token for tokens in tokenized_texts for token in tokens]

vocab = sorted(list(set(flattened_list)))
word_to_id = {word:i+1 for i, word in enumerate(vocab)}
id_to_word = {i+1:word for i, word in enumerate(vocab)}

le = LabelEncoder()
training_df['Labels']  = le.fit_transform(training_df[2])
output_y = training_df['Labels'].tolist()

73996


In [ ]:
# Read glove embeddings
glove_path = '/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt'

with open(glove_path, 'r') as file:
     lines = file.readlines()

print(len(lines))

In [ ]:
word_to_vec = dict()
for line in lines:
    word_and_vec = line.split(' ', maxsplit=1)
    word, vec = word_and_vec[0], word_and_vec[1]
    vec_array = np.fromstring(vec, sep=' ').astype('float32')
    word_to_vec[word] = vec_array


In [ ]:
input_x = ['I loved the movie', 'The product exceeded my expectations', 'The service was terrible', 'The performance of the device is disappointing']
output_y = [1, 1, 0, 0]

input_x = list(map(str.lower, input_x))
tokenized_inputs = [tokenizer(text) for text in input_x]
flattened_list = sum(tokenized_inputs, [])
print(flattened_list)
vocab = sorted(list(set(flattened_list)))
word_to_id = {word:i+1 for i, word in enumerate(vocab)}
id_to_word = {i+1:word for i, word in enumerate(vocab)}

In [7]:
encode_text = lambda x: [word_to_id[_] for _ in x]

encoded_inputs = list(map(encode_text, tokenized_texts))
padded = pad_sequence(list(map(torch.tensor, encoded_inputs)), batch_first=True)
output_y = torch.tensor(output_y, dtype=torch.float32).unsqueeze(-1)
padded.shape, output_y.shape

(torch.Size([73996, 311]), torch.Size([73996, 1]))

In [8]:
B, T = padded.shape
mask = torch.eq(padded, 0).to(torch.float32)
mask = mask * -1e9
masked_reshape = mask.reshape(B, 1, T)

In [9]:
class Embedding(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        self.embedding_layer = nn.Embedding(n_vocab, n_embed)
        
    def forward(self, x):
        return self.embedding_layer(x)

In [10]:
class Head(nn.Module):
    def __init__(self, head_size=16):
        super().__init__()
        self.query = nn.Linear(n_embed, head_size)
        self.key = nn.Linear(n_embed, head_size)
        self.value = nn.Linear(n_embed, head_size)
        
    def forward(self, x, mask):
        B, T, C = x.shape
        
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)
        
        wei = query @ key.transpose(-2, -1)
        
        if mask is not None:
            wei = wei + mask
        
        wei = F.softmax(wei, dim=-1)
        out = wei @ value # (B, T, head_size)
        
        return out

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [12]:
class Encoder(nn.Module):
    def __init__(self, n_vocab, n_embed, timesteps, head_size, output):
        super().__init__() # What happens if I pass the class name in super?
        self.embedding = Embedding(n_vocab, n_embed)
        self.positional_encodings = self.get_positional_embeddings(self.get_angle(timesteps, n_embed)).to(device)
        self.sa = Head(head_size)
        self.inter1_layer = nn.Linear(head_size, timesteps)
        self.output = nn.Linear(timesteps**2, output)
        
    def forward(self, x, mask):
        B, T = x.shape
        embedding = self.embedding(x) + self.positional_encodings # (B, timesteps, n_embed)
        sa_out = self.sa(embedding, mask) # (B, timesteps, head_size)
        inter1 = self.inter1_layer(sa_out) # (B, timesteps, head_size) @ (head_size, timesteps) --> (B, timesteps, timesteps)
        output = self.output(inter1.view(B, -1))
#         output = F.softmax(output, dim=-1)
        
        return output
    
    def get_angle(self, timesteps, dim):
        k = np.arange(dim)[np.newaxis, :]
        i = k // 2
        
        positions = np.arange(timesteps)[:, np.newaxis]
        angles = positions / (10000 ** (2*i/dim))
        
        return angles
    
    def get_positional_embeddings(self, angles):
        angles[:, 0::2] = np.sin(angles[:, 0::2])
        angles[:, 1::2] = np.cos(angles[:, 1::2])
        
        return torch.tensor(angles, dtype=torch.float32)

In [24]:
training_df[2].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [14]:
y_batch.view(-1).shape

NameError: name 'y_batch' is not defined

In [28]:
n_embed = 64
timesteps = padded.shape[-1]
model = Encoder(len(vocab) + 1, n_embed, timesteps, head_size=16, output=4)
batch_size = 32
batch_per_epoch = padded.shape[0] // batch_size

# loss_function = nn.BCELoss()
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.001 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
output_y = output_y.type(torch.LongTensor)
inputs, targets, masked_reshape = padded.to(device), output_y.to(device), masked_reshape.to(device)

for epoch in range(10):
    for i in range(batch_per_epoch):
        start = i * batch_size
        x_batch, y_batch, mask = inputs[start:start+batch_size], targets[start:start+batch_size], masked_reshape[start:start+batch_size]

        model.train()

        optimizer.zero_grad()
        output = model(x_batch, mask)
#         print(f'Shape of output from model is {output.shape} and {y_batch.shape}')
        loss = loss_function(output, y_batch.view(-1))
        print(f'Epoch {epoch}: {loss}')
        loss.backward()
        optimizer.step()
        break

Epoch 0: 1.3055124282836914
Epoch 1: 5.789517402648926
Epoch 2: 0.9687484502792358
Epoch 3: 0.2376822680234909
Epoch 4: 0.007536314427852631
Epoch 5: 1.0610582828521729
Epoch 6: 0.2768453061580658
Epoch 7: 0.019311415031552315
Epoch 8: 5.178110882297915e-07
Epoch 9: 0.0


In [ ]:
nested_list = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flattened_list = sum(nested_list, [])
print(flattened_list)
